<a href="https://colab.research.google.com/github/julialukomska70/JuliaLukomska-DataScience-GenAI-submission/blob/main/6_02_DNN_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 6.02 Dense Neural Network (with PyTorch)
This will expand on our logistic regression example and take us through building our first neural network. If you haven't already, be sure to check (and if neccessary) switch to GPU processing by clicking Runtime > Change runtime type and selecting GPU. We can test this has worked with the following code:

In [ ]:
import torch

# Check for GPU availability
print("Num GPUs Available: ", torch.cuda.device_count())

Num GPUs Available:  1


Hopefully your code shows you have 1 GPU available! Next let's get some data. We'll start with another in-built dataset:

In [ ]:
# upload an in-built Python (OK semi-in-built) dataset
from sklearn.datasets import load_diabetes

import pandas as pd
import numpy as np

# import the data
data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

We are working on a regression problem, with "structured" data which has already been cleaned and normalised. We can skip the usual cleaning/engineering steps. However, we do need to get the data into PyTorch:

In [ ]:
# Convert data to PyTorch tensors
X = torch.tensor(data.data, dtype=torch.float32)
y = torch.tensor(data.target, dtype=torch.float32).reshape(-1, 1) # Reshape y to be a column vector

Now our data is stored in tensors we can do train/test splitting as before (in fact we can use sklearn as before):

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([353, 10]) torch.Size([353, 1])
torch.Size([89, 10]) torch.Size([89, 1])


Now we can set up our batches for training. As we have a nice round 400 let's go with batches of 50 (8 batches in total). We'll also seperate the features and labels:

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

Now its time to build our model. We'll keep it simple ... a model with an input layer of 10 features and then 2x _Dense_ (fully connected) layers each with 5 neurons and ReLU activation. Our output layer will be size=1 given this is a regression problem and we want a single value output per prediction.

This will be easier to understand if you have read through the logistic regression tutorial.

In [ ]:
import torch
import torch.nn as nn

# Define the model
class DiabetesModel(nn.Module):
    def __init__(self):
        super(DiabetesModel, self).__init__()
        # we'll set up the layers as a sequence using nn.Sequential
        self.layers = nn.Sequential(

            # first layer will be a linear layer that has 5x neurons
            # (5x sets of linear regression)
            # the layer takes the 10 features as input (i.e. 10, 5)
            nn.Linear(10, 5),

            nn.ReLU(), # ReLU activation

            # second linear layer again has 5 neurons
            # this time taking the input as the output of the last layer
            # (which had 5x neurons)
            nn.Linear(5, 5),

            nn.ReLU(), # ReLU again

            # last linear layer takes the output from the previous 5 neurons
            # this time its a single output with no activation
            # i.e. this is the predicitons (regression)
            nn.Linear(5, 1)
        )

    def forward(self, x):
        return self.layers(x) # pass the data through the layers

As before we need to create a model object, specify the loss (criterion) and an optimiser (which we cover next week):

In [ ]:
import torch.optim as optim

# Initialize the model, loss function, and optimizer
model = DiabetesModel()
criterion = nn.MSELoss() # MSE loss function
optimiser = optim.Adam(model.parameters(), lr=0.001)

Now we can train the model. Again, the logistic regression tutorial (6.01) may help you undertstand this:

We can see loss is significantly lower at the end than it was at the start. However, it is also bouncing around a little still which suggests the model needs more training (100 epochs is not a lot in deep learning terms). However, let's evaluate as before:

In [ ]:
import numpy as np
import torch

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluation (example)
model.eval() # testing mode
mse_values = [] # collect the MSE scores

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs) # predict the test data

        # Calculate Mean Squared Error
        mse = criterion(outputs, targets) # calcualte mse for the batch
        mse_values.append(mse.item()) # add to the list of MSE values

# Calculate and print the average MSE
avg_mse = np.mean(mse_values)
print(f"Average MSE on test set: {avg_mse}")

Average MSE on test set: 2892.9239501953125


MSE looks expected given training (no obvious sign of overfitting). However, we probably can get better results with tuning and more epochs.

Let's run the loop again a little differently to collect the predicted values (y_hat) and actuals (y) and add them to a dataset for comparions:

In [ ]:
# Evaluation
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(targets.cpu().numpy())

# Create DataFrame
results_df = pd.DataFrame({'Predicted': np.array(predictions).flatten(), 'Actual': np.array(actuals).flatten()})
results_df

,Predicted,Actual
0,156.549942,219.0
1,169.873535,70.0
2,149.057144,202.0
3,288.014343,230.0
4,137.688431,111.0
...,...,...
84,119.575607,153.0
85,90.198990,98.0
86,73.640121,37.0
87,70.573914,63.0


Side-by-side, they don't look great. Can you improve them?

<br><br>

## EXERCISE #1
Try increasing the number of epochs to 1,000 (when the model is fairly well trained then the results printed for each 10x epochs will be fairly stable and not change much). Does this give better results?

<br><br>

## EXERCISE #2 (optional)
Try experimenting with the architecture (number of neurons and/or number of layers). Can we reach an optimal architecture?

In [ ]:
import torch.optim as optim

# Initialize the model, loss function, and optimizer
model = DiabetesModel()
criterion = nn.MSELoss() # MSE loss function
optimiser = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop (example - you'll likely want to add more epochs)
epochs = 1000 # 1000 epochs

for epoch in range(epochs):
  # use the train_loader to pass the inputs (x) and targets (y)
  for inputs, targets in train_loader:
    # pass to the GPU (hopefully)
    inputs, targets = inputs.to(device), targets.to(device)

    # pass model to GPU as well
    model.to(device)

    model.train() # put the model object in train mode
    optimiser.zero_grad() # reset the gradiants
    outputs = model(inputs) # create outputs
    loss = criterion(outputs, targets) # compare with Y to get loss
    loss.backward() # backpropogate the loss (next week)
    optimiser.step() # # update the parameters based on this round of training

  # every 10 steps we will print out the current loss
    if (epoch+1) % 10 == 0: # modular arithmetic
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {round(loss.item(), 4)}')

Epoch [10/1000], Loss: 25718.6074
Epoch [10/1000], Loss: 28578.5898
Epoch [10/1000], Loss: 30003.4277
Epoch [10/1000], Loss: 31593.6367
Epoch [10/1000], Loss: 32739.7461
Epoch [10/1000], Loss: 26698.6348
Epoch [10/1000], Loss: 32083.4297
Epoch [10/1000], Loss: 27132.6465
Epoch [20/1000], Loss: 30674.2266
Epoch [20/1000], Loss: 24989.834
Epoch [20/1000], Loss: 29398.3145
Epoch [20/1000], Loss: 32127.9043
Epoch [20/1000], Loss: 24140.5312
Epoch [20/1000], Loss: 31930.4824
Epoch [20/1000], Loss: 31891.0547
Epoch [20/1000], Loss: 44064.375
Epoch [30/1000], Loss: 31140.0117
Epoch [30/1000], Loss: 32023.4785
Epoch [30/1000], Loss: 25831.8828
Epoch [30/1000], Loss: 32945.3086
Epoch [30/1000], Loss: 28723.0234
Epoch [30/1000], Loss: 26577.0195
Epoch [30/1000], Loss: 24926.7539
Epoch [30/1000], Loss: 60355.2188
Epoch [40/1000], Loss: 29591.8652
Epoch [40/1000], Loss: 31496.6797
Epoch [40/1000], Loss: 27497.7539
Epoch [40/1000], Loss: 32137.8945
Epoch [40/1000], Loss: 27523.0098
Epoch [40/1000],

Summary:

GPU Availability: The environment successfully detected and utilized 1 GPU for training.
Data Preparation: The Diabetes dataset was loaded, converted to PyTorch tensors, split into training and testing sets, and batched using TensorDataset and DataLoader.
Model Architecture: A simple dense neural network was defined with an input layer, two hidden layers (5 neurons each with ReLU activation), and a single-neuron output layer for regression.
Initial Training & Evaluation (100 Epochs): When first run with 100 epochs, the model yielded an average Mean Squared Error (MSE) on the test set of approximately 2892.92.
Increased Epochs (1000 Epochs): After increasing the number of training epochs to 1000, the model was retrained. The average MSE on the test set remained approximately 2892.92. This indicates that simply increasing the number of epochs alone did not lead to a significant improvement in the model's performance in this instance. The loss during training showed a general decreasing trend but stabilized, suggesting the model may have converged or reached a local minimum with the current architecture and hyperparameters.
Predictions: A comparison of predicted versus actual values on the test set was shown, and they didn't look ideal, reinforcing the idea that there's room for improvement.

# Task
Modify the `DiabetesModel` class in cell `844H60hcCV3s` to add an extra hidden layer and increase the number of neurons in the existing hidden layers.

## Modify Model Architecture

### Subtask:
Modify the `DiabetesModel` class in cell `844H60hcCV3s` to experiment with the number of neurons and/or layers. For example, we can add an extra hidden layer and increase the number of neurons in the existing hidden layers.


**Reasoning**:
To experiment with the model architecture as requested, I will modify the `DiabetesModel` class in cell `844H60hcCV3s` by adding an extra hidden layer and increasing the number of neurons in the existing hidden layers.



In [ ]:
import torch
import torch.nn as nn

# Define the model
class DiabetesModel(nn.Module):
    def __init__(self):
        super(DiabetesModel, self).__init__()
        # we'll set up the layers as a sequence using nn.Sequential
        self.layers = nn.Sequential(

            # first layer will be a linear layer that has 16x neurons
            # (16x sets of linear regression)
            # the layer takes the 10 features as input (i.e. 10, 16)
            nn.Linear(10, 16),

            nn.ReLU(), # ReLU activation

            # second linear layer again has 16 neurons
            # this time taking the input as the output of the last layer
            # (which had 16x neurons)
            nn.Linear(16, 16),

            nn.ReLU(), # ReLU again

            # New third linear layer with 16 neurons
            nn.Linear(16, 16),

            nn.ReLU(), # ReLU activation for the new layer

            # last linear layer takes the output from the previous 16 neurons
            # this time its a single output with no activation
            # i.e. this is the predicitons (regression)
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x) # pass the data through the layers

## Retrain Model

### Subtask:
Retrain the model with the modified architecture using the training loop in cell `PtMUgfwT-HGt`. This will involve rerunning the cell to compile and train the updated model.


**Reasoning**:
The subtask requires retraining the model with the modified architecture. Cell `PtMUgfwT-HGt` contains the training loop and model initialization, so executing this cell will achieve the subtask.



In [ ]:
import torch.optim as optim

# Initialize the model, loss function, and optimizer
model = DiabetesModel()
criterion = nn.MSELoss() # MSE loss function
optimiser = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop (example - you'll likely want to add more epochs)
epochs = 1000 # 1000 epochs

for epoch in range(epochs):
  # use the train_loader to pass the inputs (x) and targets (y)
  for inputs, targets in train_loader:
    # pass to the GPU (hopefully)
    inputs, targets = inputs.to(device), targets.to(device)

    # pass model to GPU as well
    model.to(device)

    model.train() # put the model object in train mode
    optimiser.zero_grad() # reset the gradiants
    outputs = model(inputs) # create outputs
    loss = criterion(outputs, targets) # compare with Y to get loss
    loss.backward() # backpropogate the loss (next week)
    optimiser.step() # # update the parameters based on this round of training

  # every 10 steps we will print out the current loss
    if (epoch+1) % 10 == 0: # modular arithmetic
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {round(loss.item(), 4)}')

Epoch [10/1000], Loss: 29896.5859
Epoch [10/1000], Loss: 30151.9316
Epoch [10/1000], Loss: 36237.1875
Epoch [10/1000], Loss: 26645.6445
Epoch [10/1000], Loss: 25566.5586
Epoch [10/1000], Loss: 24897.4766
Epoch [10/1000], Loss: 34496.3125
Epoch [10/1000], Loss: 24726.9688
Epoch [20/1000], Loss: 32060.3672
Epoch [20/1000], Loss: 28220.0977
Epoch [20/1000], Loss: 32637.2637
Epoch [20/1000], Loss: 23175.5625
Epoch [20/1000], Loss: 32391.7949
Epoch [20/1000], Loss: 27256.2246
Epoch [20/1000], Loss: 28247.5293
Epoch [20/1000], Loss: 7871.9756
Epoch [30/1000], Loss: 21545.25
Epoch [30/1000], Loss: 23765.7695
Epoch [30/1000], Loss: 28638.0527
Epoch [30/1000], Loss: 23551.4336
Epoch [30/1000], Loss: 29581.6035
Epoch [30/1000], Loss: 19033.3047
Epoch [30/1000], Loss: 22150.0176
Epoch [30/1000], Loss: 21387.0332
Epoch [40/1000], Loss: 14219.4043
Epoch [40/1000], Loss: 11366.457
Epoch [40/1000], Loss: 11349.3994
Epoch [40/1000], Loss: 8859.3965
Epoch [40/1000], Loss: 11754.6172
Epoch [40/1000], Lo

**Reasoning**:
The model has been retrained with the modified architecture. Now, to evaluate its performance, I need to run the evaluation code to calculate the Mean Squared Error (MSE) on the test set and generate the DataFrame of predicted vs actual values. This will allow for a comparison of the new model's performance against the previous iteration.



In [ ]:
import numpy as np
import torch

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluation (example)
model.eval() # testing mode
mse_values = [] # collect the MSE scores

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs) # predict the test data

        # Calculate Mean Squared Error
        mse = criterion(outputs, targets) # calcualte mse for the batch
        mse_values.append(mse.item()) # add to the list of MSE values

# Calculate and print the average MSE
avg_mse = np.mean(mse_values)
print(f"Average MSE on test set: {avg_mse}")

Average MSE on test set: 2797.614013671875


**Reasoning**:
The subtask requires evaluating the model with the modified architecture by comparing predicted values with actual values and storing them in a DataFrame for comparison. The `results_df` in the kernel state was generated from the previous model. So, I will rerun the evaluation code to generate the predicted and actual values and store them in a DataFrame.



In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(targets.cpu().numpy())

# Create DataFrame
results_df = pd.DataFrame({'Predicted': np.array(predictions).flatten(), 'Actual': np.array(actuals).flatten()})
results_df

,Predicted,Actual
0,141.356094,219.0
1,180.991943,70.0
2,140.620941,202.0
3,298.889435,230.0
4,118.210754,111.0
...,...,...
84,109.386101,153.0
85,87.415108,98.0
86,84.832199,37.0
87,69.315536,63.0


## Final Task

### Subtask:
Compare the results of the new architecture with the previous attempts and summarize whether the changes led to an improved performance.


## Summary:

### Q&A
The task was to compare the results of the new architecture with previous attempts and summarize whether the changes led to improved performance.
Based on the provided information, the average Mean Squared Error (MSE) on the test set for the new architecture is 2797.61. However, the performance metrics of previous attempts are not provided in the current context, so a direct comparison to determine if performance improved cannot be made.

### Data Analysis Key Findings
*   The `DiabetesModel` architecture was successfully modified to include an additional hidden layer and increase the number of neurons in all hidden layers to 16.
*   The modified model was retrained for 1000 epochs, during which the training loss consistently decreased, indicating the model was learning.
*   Upon evaluation on the test set, the retrained model achieved an average Mean Squared Error (MSE) of approximately 2797.61.
*   A DataFrame was generated that compares the model's predicted values against the actual values for the test set, which can be used for further detailed analysis.

### Insights or Next Steps
*   To fully assess the impact of the architectural changes, compare the obtained MSE of 2797.61 with the MSE from previous model architectures.
*   Further hyperparameter tuning (e.g., learning rate, number of epochs) or exploring other architectural variations could potentially lead to improved performance.
